# Finetuning

Based on https://www.youtube.com/watch?v=CbmTFTsbyPI

https://huggingface.co/openlm-research/open_llama_3b_v2

Need to install the transformers library to run this notebook:
```bash
conda install pytorch-nightly::pytorch torchvision torchaudio -c pytorch-nightly
conda install -c huggingface transformers  
conda install -c conda-forge sentencepiece peft accelerate 
```

In [1]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import torch
import random

random.seed(42)
torch.manual_seed(42)

In [2]:
device = 'cpu'
if torch.cuda.is_available():
	device = 'cuda' 
elif torch.backends.mps.is_available():
	device = 'mps'
print(f'using device: {device}')

using device: mps


In [3]:
from transformers import LlamaTokenizer, LlamaForCausalLM
model_path = 'openlm-research/open_llama_3b_v2'
tokenizer = LlamaTokenizer.from_pretrained(model_path, legacy=True)
base_model = LlamaForCausalLM.from_pretrained(model_path)

/Users/cwd7/miniconda3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from peft import LoraConfig, PeftModel
lora_config = LoraConfig(
	r=64,
	lora_alpha=32,
	lora_dropout=0.05,
	bias="none",
	task_type="CAUSAL_LM",
)
model = PeftModel(base_model, lora_config, adapter_name="Shakespeare")

device=torch.device(device)
model.to(device)

PeftModel(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 3200, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3200, out_features=3200, bias=False)
                (lora_dropout): ModuleDict(
                  (Shakespeare): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (Shakespeare): Linear(in_features=3200, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (Shakespeare): Linear(in_features=64, out_features=3200, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(in_features=3200, out_features=3200, bi

In [5]:
import os
import requests
file_name = "input.txt"

from transformers import TextDataset
train_dataset = TextDataset(
	tokenizer=tokenizer,
	file_path=file_name,
	block_size=128,
)[:256]

/Users/cwd7/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [6]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
	output_dir="./output",
	overwrite_output_dir=True,
	num_train_epochs=10,
	per_device_train_batch_size=32,
	eval_strategy="no",
)

In [7]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(
	tokenizer=tokenizer, mlm=False,
)
trainer = Trainer(
	model=model,
	args=training_args,
	data_collator=data_collator,
	train_dataset=train_dataset,
)

In [8]:
def generate_response(prompt_text, model, tokenizer, max_length=30, num_return_sequences=1):
	# input_ids = tokenizer.encode(prompt_text, return_tensors="pt").to(device)
	# Don't have mps support at this time
	input_ids = tokenizer.encode(prompt_text, return_tensors="pt").to('cpu')
	outputs_sequences = model.generate(
		input_ids=input_ids,
		max_length=max_length,
		num_return_sequences=num_return_sequences,
		no_repeat_ngram_size=2,
	)

	responses = []
	for response_id in outputs_sequences:
		response = tokenizer.decode(response_id, skip_special_tokens=True)
		responses.append(response)
	return responses

In [10]:
prompt_text = "Uneasy lies the head that wears a crown"

responses = generate_response(prompt_text, model, tokenizer)
for response in responses:
	print(response)

ValueError: Can't infer missing attention mask on `mps` device. Please provide an `attention_mask` or use a different device.

In [ ]:
trainer.train()